In [1]:
import threading
import multiprocessing
import numpy as np
import tensorflow as tf
from AtariEnvironment import *
from A3CNetwork import * 
#from ReplayBuffer import *
from Trainer import train,train_acer, test
import gym as gym
from time import sleep
from time import time
from utils import preprocess


GAME_NAME="Breakout-v0"
#GAME_NAME = "LunarLander-v2"
TB_DIR = "saves/breakoutA3C/2m4/"
env=Atari_Environment(gym.make(id=GAME_NAME))
#env = gym.make(GAME_NAME)
state= env.reset()
training_finished = False
LEARNING_RATE = 7e-4

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [2]:
print(env.env.unwrapped.frameskip)

(2, 5)


In [3]:
session = tf.Session()
tf.reset_default_graph()

with tf.Session() as sess:
    with tf.device('/cpu:0'):
        G_Agent=A3CNetwork(sess,env,"global",network_type="cnn")
        agent = A3CNetwork(sess,env,str(0),G_Agent, network_type="cnn")
        sess.run(tf.global_variables_initializer())
        print(agent.get_pi(state))
        print(G_Agent.get_pi(state))
        agent.update_target()
        print(agent.get_pi(state))
        print(G_Agent.get_pi(state))


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


[[0.24992192 0.24890877 0.2514087  0.24976066]]
[[0.24949715 0.24992645 0.25057748 0.24999893]]
[[0.24949715 0.24992645 0.25057748 0.24999893]]
[[0.24949715 0.24992645 0.25057748 0.24999893]]


In [4]:
def start_method(game_name=GAME_NAME, num_agents = 16, restore = False, save_path=TB_DIR):
        
    envs=[]
    agents=[]
    processes=[]
    
    for _ in range(num_agents):
        #a_env = gym.make(game_name)
        #a_env = Custom_Environment(gym.make(game_name))
        a_env = Atari_Environment(gym.make(game_name), clipreward=True)
        envs.append(a_env)
       

    with tf.Session() as sess:
        with tf.device('/cpu:0'):
            G_Agent=A3CNetwork(sess,envs[0],"global",network_type="cnn",lr=LEARNING_RATE)
            for agent_num in range(num_agents):
                agents.append(A3CNetwork(sess,envs[0],str(agent_num),G_Agent,network_type="cnn"))
            saver = tf.train.Saver(max_to_keep=5)

            if restore:
                ckpt = tf.train.get_checkpoint_state(save_path+"checkpoints")
                saver.restore(sess,ckpt.model_checkpoint_path)
            else:
                sess.run(tf.global_variables_initializer())
            # Create a process for each worker
            for thread_id in range(num_agents):
                processes.append(threading.Thread(target=train, args=(agents[thread_id],
                envs[thread_id], sess, thread_id, 20, 0.99, 500000, 200, saver, save_path)))
            # Start all the processes
            for p in processes:
                p.daemon = True
                p.start()

            # Until training is finished
            while not training_finished:
                sleep(0.05)

            # Join the processes, so we get this thread back.
            for p in processes:
                p.join()


In [ ]:
tf.reset_default_graph()
start_method()

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Starting AgentStarting Agent 1
 Starting Agent0
 2
Starting AgentStarting Agent 4
 3
Starting Agent 5
Starting Agent 6
Starting Agent 7
Starting Agent 8
Starting Agent Starting Agent 10
9
Starting Agent 11
Starting Agent 12
Starting Agent 13
Starting Agent Starting Agent 15
14


/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Worker  14 At  200  Running/Max:  0.0 0.0
Worker  10 At  400  Running/Max:  0.5 5.0
Worker  0 At  600  Running/Max:  0.3520000000000001 2.0
Worker  11 At  800  Running/Max:  0.552 3.0
Worker  6 At  1000  Running/Max:  0.32580000000000003 2.0
[[0.25005838 0.25010997 0.24988072 0.24995099]] 0.42277366
Worker  3 At  1200  Running/Max:  0.5678300000000001 3.0
Worker  12 At  1400  Running/Max:  0.950698 4.0
Worker  13 At  1600  Running/Max:  0.6721893000000001 3.0
Worker  14 At  1800  Running/Max:  0.697862953 3.0
Worker  4 At  2000  Running/Max:  0.736094791 3.0
[[0.24993646 0.25012994 0.2501392  0.24979445]] 0.9647996
Worker  11 At  2200  Running/Max:  0.6208362799200001 3.0
Worker  15 At  2400  Running/Max:  0.9095697389000001 3.0
Saving Model
Worker  15 At  2600  Running/Max:  1.3186127650100001 5.0
Worker  0 At  2800  Running/Max:  1.1714233369152003 6.0
Worker  14 At  3000  Running/Max:  0.901431985605273 3.0
[[0.24997774 0.25001678 0.25000858 0.24999692]] 0.2542017
Worker  8 At  3200

Worker  8 At  23800  Running/Max:  1.373486427215911 4.0
Worker  14 At  24000  Running/Max:  1.1652080028192453 6.0
[[0.2908136  0.21693903 0.27989772 0.21234962]] 0.361362
Worker  15 At  24200  Running/Max:  2.0553520771646507 7.0
Worker  1 At  24400  Running/Max:  1.8738244798885102 5.0
Worker  1 At  24600  Running/Max:  1.6864420318996591 5.0
Worker  3 At  24800  Running/Max:  1.975269414719016 6.0
Worker  13 At  25000  Running/Max:  1.6110510884876932 5.0
Saving Model
[[0.33649686 0.19400378 0.26273745 0.2067619 ]] 0.20014924
Worker  7 At  25200  Running/Max:  1.7278847513548876 5.0
Worker  13 At  25400  Running/Max:  1.8549513816750316 5.0
Worker  1 At  25600  Running/Max:  1.9967746171293665 7.0
Worker  2 At  25800  Running/Max:  1.6631323585203701 4.0
Worker  9 At  26000  Running/Max:  1.67287279298301 5.0
[[0.30140373 0.22288236 0.25888366 0.21683034]] 0.3741756
Worker  3 At  26200  Running/Max:  1.9759571530276892 6.0
Worker  2 At  26400  Running/Max:  1.91742348936135 5.0
Wor

Worker  0 At  47200  Running/Max:  1.4467961674939098 6.0
Worker  5 At  47400  Running/Max:  1.4438697635643813 5.0
Saving Model
Worker  2 At  47600  Running/Max:  0.7496398906437244 6.0
Worker  0 At  47800  Running/Max:  1.4157144061030604 6.0
Worker  14 At  48000  Running/Max:  1.2130349122155744 6.0
[[0.31411937 0.19091713 0.2281374  0.26682612]] 0.2834193
Worker  7 At  48200  Running/Max:  1.1591613255083544 6.0
Worker  13 At  48400  Running/Max:  1.6615363950872262 5.0
Worker  12 At  48600  Running/Max:  1.3181220436054784 5.0
Worker  8 At  48800  Running/Max:  0.9624647511301611 7.0
Worker  2 At  49000  Running/Max:  0.7861584358112323 6.0
[[0.30843806 0.18929198 0.2320572  0.27021274]] 0.27741742
Worker  1 At  49200  Running/Max:  1.696028749616404 7.0
Worker  4 At  49400  Running/Max:  0.878601982268582 6.0
Worker  3 At  49600  Running/Max:  1.6409040491956224 6.0
Worker  6 At  50000  Running/Max:  0.7349001844532377 6.0
Saving Model
[[0.29084992 0.19148386 0.24090706 0.2767591

In [ ]:
def test_method(game_name=GAME_NAME, num_agents = 1, restore = True, save_path=TB_DIR):
        
    envs=[]
    agents=[]
    processes=[]
    
    for _ in range(num_agents):
        #a_env = gym.make(game_name)
        #a_env = Custom_Environment(gym.make(game_name))
        a_env = Atari_Environment(gym.make(game_name), clipreward=True)
        envs.append(a_env)
       


    with tf.Session() as sess:
        with tf.device('/cpu:0'):
            G_Agent=ACERNetwork(sess,envs[0],"global",network_type="cnn",lr=LEARNING_RATE)
            for agent_num in range(num_agents):
                agents.append(ACERNetwork(sess,envs[0],str(agent_num),G_Agent,network_type="cnn"))
            saver = tf.train.Saver(max_to_keep=2)

            if restore:
                ckpt = tf.train.get_checkpoint_state(save_path+"checkpoints")
                saver.restore(sess,ckpt.model_checkpoint_path)
            else:
                sess.run(tf.global_variables_initializer())
            # Create a process for each worker
            for thread_id in range(num_agents):
                processes.append(threading.Thread(target=test, args=(agents[thread_id],
                envs[thread_id], 2, True)))
            # Start all the processes
            for p in processes:
                p.daemon = True
                p.start()

            # Until training is finished
            while not training_finished:
                sleep(0.05)

            # Join the processes, so we get this thread back.
            for p in processes:
                p.join()


In [ ]:
tf.reset_default_graph()
test_method()